In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 26247883
paper_name = 'ghavidel_harkness_2015' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/hits.xlsx', sheet_name='Sheet1', header=None)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 52 x 3


In [10]:
original_data.head()

,0,1,2
0,YAL013W,-1.16,-3.99
1,YCL001W,-1.01,-3.64
2,YPR037C,-2.04,-3.94
3,YBR084C-A,-0.89,-3.84
4,YGR227W,-1.06,-3.66


In [11]:
original_data['orf'] = original_data[0].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [0, 1, 2, orf]
Index: []


In [15]:
original_data.set_index('orf', inplace=True)

In [19]:
original_data = -original_data[[1,2]].copy()

In [20]:
original_data = original_data.groupby(original_data.index).mean()

In [21]:
original_data.shape

(52, 2)

# Prepare the final dataset

In [22]:
data = original_data.copy()

In [24]:
dataset_ids = [4923, 11858]
datasets = datasets.reindex(index=dataset_ids)

In [25]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [26]:
data.head()

dataset_id,4923,11858
data_type,value,value
orf,,
YAL013W,1.16,3.99
YAL028W,1.47,2.87
YAR002C-A,2.87,3.18
YAR040C,0.72,2.32
YBR084C-A,0.89,3.84


## Subset to the genes currently in SGD

In [27]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 1


In [28]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,4923,11858
,data_type,value,value
gene_id,orf,,
11,YAL013W,1.16,3.99
26,YAL028W,1.47,2.87
1859,YAR002C-A,2.87,3.18
1877,YBR084C-A,0.89,3.84
309,YBR115C,1.05,2.52


# Normalize

In [29]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [30]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [31]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,4923,11858,4923,11858
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
11,YAL013W,1.16,3.99,7.416414,12.452240
26,YAL028W,1.47,2.87,9.398387,8.956874
1859,YAR002C-A,2.87,3.18,18.349232,9.924342
1877,YBR084C-A,0.89,3.84,5.690180,11.984111
309,YBR115C,1.05,2.52,6.713134,7.864573


# Print out

In [32]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [33]:
from IO.save_data_to_db3 import *

In [34]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 26247883...
Inserting the new data...


100%|██████████| 2/2 [00:00<00:00,  6.59it/s]


Updating the data_modified_on field...
